In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('../dataset/transform/data100.csv')
data = data.fillna(0)
data.head()

,modulation,timeDomain_1,timeDomain_2,timeDomain_3,timeDomain_4,timeDomain_5,timeDomain_6,timeDomain_7,timeDomain_8,timeDomain_9,...,cs_prior__1021,cs_prior__1022,cs_prior__1023,cs_prior__1024,sDomain_max,sDomain_min,sDomain_mean,sDomain_1st_Qu,sDomain_2nd_Qu,sDomain_3rd_Qu
0,8PSK,0.639289,-0.213478,-0.865621,0.386868,0.621987,-0.567360,0.010030,0.373295,-0.150273,...,-0.939505,-0.937342,-1.158446,-1.151277,38.690961,0.037595,5.076279,0.955909,1.621237,7.163684
1,16QAM,-0.343101,-0.515628,-0.672854,0.547040,1.419239,-0.167272,-1.138051,-0.069713,1.464533,...,-1.620745,-1.181503,-1.134026,-0.515628,47.539106,0.000846,5.453533,1.893632,3.143240,5.845227
2,64QAM,-1.054600,-0.523152,0.886420,0.546905,-0.485738,-0.524877,-0.020109,0.228228,0.193880,...,-0.590973,-1.065564,-1.218445,-0.738838,38.200695,0.000383,4.535727,0.325565,0.591289,6.332244
3,BPSK,0.201801,0.990348,-0.083518,-0.947808,0.114776,0.990819,-0.046717,-1.288954,0.392954,...,-0.754663,-0.737908,-0.806254,-0.696160,37.163558,0.006632,4.472471,0.413386,0.768464,5.450173
4,CPFSK,0.168200,0.239790,-0.023122,-0.187393,-0.080768,0.208151,0.165164,-0.273555,-0.237533,...,-1.300745,-1.395009,-1.410663,-1.383565,59.125770,0.056253,4.774119,0.993590,1.838552,4.677182


In [3]:
#prefix_list = ['zeroCross_X_', 'zeroCross_Y_', 'cs_current_', 'cs_prior_']
#l = []
#l.extend(range(1, 1024))
#resultDrop = ['{}_{}'.format(a, b) for b in l for a in prefix_list]
 
#s_ww_columns = ("sDomain_max", "sDomain_min" ,"sDomain_mean", "sDomain_1st_Qu" , "sDomain_2nd_Qu", "sDomain_3rd_Qu")
#for x in s_ww_columns:
#    resultDrop.append(x)
#resultDrop.append("modulation")

y = data.modulation
#X = data.drop(columns = resultDrop).to_numpy()
X = data.drop(columns = "modulation").to_numpy()

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

algorithms = {
    "RF": (RandomForestClassifier(), { "criterion": ["gini", "entropy"], "max_depth": [5, 10, 100], "n_estimators": [5, 10, 100, 1000] }),
    "DT" : (DecisionTreeClassifier(), { "criterion": ["gini", "entropy"], "max_depth": [5, 10, 100] })
}

In [5]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, GridSearchCV, KFold
from sklearn.metrics import accuracy_score, make_scorer, balanced_accuracy_score

#kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=17) 
kf = KFold(n_splits=10, shuffle=True, random_state=17) 
#gskf = StratifiedKFold(n_splits=10, shuffle=True, random_state=17) 
gskf = KFold(n_splits=10, shuffle=True, random_state=17) 
perf = accuracy_score

correct = {}
for algorithm in algorithms.keys():
    correct[algorithm] = y.copy()
    
score = {}
best_param = {}
best_estimator_param = {}
for algorithm in algorithms.keys():
    score[algorithm] = []
    best_param[algorithm] = []
    best_estimator_param[algorithm] = []
    
max_score = 0
k_max_ac = 1
for algorithm, (clf, parameters) in algorithms.items():
    for train, test in kf.split(X, y):
        prep = StandardScaler()
        prep.fit(X[train])
        
        best = GridSearchCV(clf, parameters, cv=gskf, scoring="accuracy")
        best.fit(prep.transform(X[train]), y[train])
        
        predicted = best.predict(prep.transform(X[test]))
        print(balanced_accuracy_score(y[test], predicted))
        correct[algorithm].loc[test] = predicted == y[test]
        score[algorithm].append(perf(best.predict(X[test]), y[test]))
        best_param[algorithm].append(best.best_params_)
        best_estimator_param[algorithm].append(best.best_estimator_)

0.06054421768707482
0.16706349206349208
0.1455128205128205
0.13096784168212738
0.17243312243312245
0.08537660256410257
0.16397709433423718
0.13776699490985206
0.1169053169053169
0.12316849816849818
0.1447278911564626
0.1640873015873016
0.10622710622710622
0.1805040197897341
0.17519702519702518
0.14590010683760685
0.1334875442018299
0.09508348794063079


/usr/local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1814: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')


0.1595959595959596
0.17765567765567764


In [6]:
print("Mean\n", pd.DataFrame.from_dict(score).mean())
print("\nStd\n", pd.DataFrame.from_dict(score).std())

Mean
 RF    0.15875
DT    0.13375
dtype: float64

Std
 RF    0.040846
DT    0.019588
dtype: float64


In [7]:
pd.DataFrame.from_dict(correct)

,RF,DT
0,True,False
1,True,False
2,False,False
3,False,False
4,True,False
...,...,...
795,False,True
796,True,False
797,True,False
798,False,False


In [8]:
pd.DataFrame.from_dict(score)

,RF,DT
0,0.1125,0.1125
1,0.1000,0.1625
2,0.2125,0.1125
3,0.1625,0.1125
4,0.1500,0.1250
5,0.2250,0.1375
6,0.1500,0.1625
7,0.1625,0.1500
8,0.1250,0.1250
9,0.1875,0.1375


In [9]:
print("Max Score\n", pd.DataFrame.from_dict(score).max())

Max Score
 RF    0.2250
DT    0.1625
dtype: float64


In [10]:
for algorithm in algorithms.items():
    alg = algorithm[0]
    idx = pd.DataFrame.from_dict(score)[alg].idxmax()
    print("Best Parameters "+alg+" of max score", pd.DataFrame.from_dict(best_param).loc[idx, alg])

Best Parameters RF of max score {'criterion': 'gini', 'max_depth': 10, 'n_estimators': 10}
Best Parameters DT of max score {'criterion': 'gini', 'max_depth': 100}


In [11]:
for algorithm in algorithms.items():
    alg = algorithm[0]
    idx = pd.DataFrame.from_dict(score)[alg].idxmax()
    print("Best Estimator Parameters "+alg+" of max score", pd.DataFrame.from_dict(best_estimator_param).loc[idx, alg])

Best Estimator Parameters RF of max score RandomForestClassifier(max_depth=10, n_estimators=10)
Best Estimator Parameters DT of max score DecisionTreeClassifier(max_depth=100)


In [12]:
import numpy as np
from mlxtend.evaluate import mcnemar_table

In [13]:
# The correct target (class) labels
idx_modulation = {"8PSK":0, "16QAM":1, "64QAM":2, "BPSK":3, "CPFSK":4, "GFSK":5, "QPSK":6, "PAM4":7}
y_target = y.replace(idx_modulation)

# Class labels predicted by model RandomForestClassifier
y_RF = pd.DataFrame.from_dict(correct).RF

# Class labels predicted by model DecisionTreeClassifier
y_DT = pd.DataFrame.from_dict(correct).DT

tb = mcnemar_table(y_target=y_target, 
                   y_model1=y_RF, 
                   y_model2=y_DT)

print(tb)

[[ 90  26]
 [ 24 660]]


In [14]:
ct=pd.DataFrame(tb, columns=["RF positive", "RF negative"])
ct.index=["DT positive", "DT negative"] 
ct

,RF positive,RF negative
DT positive,90,26
DT negative,24,660
